# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [35]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3, controllable = False)
pp.create_load(net, bus3, p_kw=70e3, controllable = False)
pp.create_load(net, bus4, p_kw=10e3, controllable = False)

#create generators
eg = pp.create_ext_grid(net, bus1)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=-80e3, max_p_kw=0,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=-100e3, max_p_kw=0, vm_pu=1.01, controllable=True)

## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [36]:
costeg = pp.create_polynomial_cost(net, 0, 'ext_grid', np.array([1, 0]))
costgen1 = pp.create_polynomial_cost(net, 0, 'gen', np.array([1, 0]))
costgen2 = pp.create_polynomial_cost(net, 1, 'gen', np.array([1, 0]))

We run an OPF:

In [37]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Numerically failed.
Did not converge in 2 iterations.


OPFNotConverged: Optimal Power Flow did not converge!

Apparently the OPF did not converge. Sometimes the constraints are too broad or too narrow for the OPF to converge. So let's try setting the voltage constraints first:

In [38]:
net.bus['max_vm_pu'] = 2
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [39]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.133524,-1974.471614


In [40]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.545821,1969.654618,-3.712804,1.000009
1,-12299.610412,1451.160058,-3.712782,1.000010


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [41]:
net.polynomial_cost.c.at[costeg] = np.array([[0.1, 0]])
net.polynomial_cost.c.at[costeg] = np.array([[0.15, 0]])
net.polynomial_cost.c.at[costeg] = np.array([[0.12, 0]])

And now run an OPF:

In [42]:
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


We can see that all active power is provided by the external grid. This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [43]:
net.res_cost

17347.141253106995

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [44]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,144559.491768,9193.072684,-143959.562873,15993.752558,599.928895,25186.825242,0.380137,0.759989,144.851508


We now limit the transformer loading to 50%:

In [45]:
net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [46]:
pp.runopp(net)

We can see that the transformer complies with the maximum loading:

In [47]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49999.973662,50.125059,-49880.069475,2969.88545,119.904186,3020.010509,0.131216,0.262147,49.999999


And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [48]:
net.res_ext_grid

,p_kw,q_kvar
0,-49999.973662,-50.125059


In [49]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-73695.159261,1571.483085,-2.964891,1.005598
1,-16467.058365,731.990548,-2.964879,1.005598


This comes of course with an increase in dispatch costs:

In [50]:
net.res_cost

96162.214465430821

### Line Loading Constraints

Wen now look at the line loadings:

In [51]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-3679.914915,-1579.335851,3695.291550,-730.606814,1.537664e+01,-2309.942665,0.021009,0.019661,0.021009,4.469931
1,-0.132286,-840.876268,0.132286,-840.875682,1.390778e-08,-1681.751951,0.004389,0.004389,0.004389,0.933806
2,6466.926080,108.885137,-6440.015611,-1390.549598,2.691047e+01,-1281.664460,0.033758,0.034565,0.034565,7.354172


and run the OPF with a 50% loading constraint:

In [52]:
net.line["max_loading_percent"] = 50
pp.runopp(net, verbose=True)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


Now the line loading constraint is complied with:

In [53]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,-3679.939900,-1579.335840,3695.316741,-730.606457,1.537684e+01,-2309.942297,0.021009,0.019661,0.021009,4.469956
1,-0.076634,-840.876072,0.076634,-840.875899,4.665948e-09,-1681.751972,0.004389,0.004389,0.004389,0.933806
2,6466.900384,108.884960,-6439.990125,-1390.549843,2.691026e+01,-1281.664883,0.033758,0.034564,0.034564,7.354144


And all generators are involved in supplying the loads:

In [54]:
net.res_ext_grid

,p_kw,q_kvar
0,-49999.974164,-50.124886


In [55]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-73695.240106,1571.482530,-2.964888,1.005598
1,-16466.977017,731.990941,-2.964881,1.005598


This of course comes with a once again rising dispatch cost:

In [56]:
net.res_cost

96162.21402327175

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [57]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49999.974164,-50.124886
1,1.000454,-3.435518,60000.000000,0.000000
2,1.005598,-2.964888,-3695.240106,1571.482530
3,1.005598,-2.964881,-6466.977017,731.990941


and constrain it:

In [58]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net)

We can see that all voltages are within the voltage band:

In [59]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.000000,0.000000,-49999.976305,-49.335367
1,1.000455,-3.435516,60000.000000,0.000000
2,1.005599,-2.964911,-3695.131455,1571.157579
3,1.005599,-2.964895,-6467.084122,731.540081


And all generators are once again involved in supplying the loads:

In [60]:
net.res_ext_grid

,p_kw,q_kvar
0,-49999.976305,-49.335367


In [61]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-73695.131455,1571.157579,-2.964911,1.005599
1,-16467.084122,731.540081,-2.964895,1.005599


This of course comes once again with rising dispatch costs:

In [62]:
net.res_cost

96162.212733719163

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [63]:
pp.rundcopp(net)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints canot be considered in the DC OPF:

In [64]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,NaN,0.000000,-49999.999996,NaN
1,NaN,-3.436967,60000.000000,NaN
2,NaN,-3.845626,17622.600459,NaN
3,NaN,-2.426876,-27622.600463,NaN


Line and transformer loading limits are however complied with:

In [65]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_from_ka,i_to_ka,i_ka,loading_percent
0,3007.062643,NaN,-3007.062643,NaN,NaN,NaN,0.015783,0.015783,0.015783,3.358082
1,-14615.537817,NaN,14615.537817,NaN,NaN,NaN,0.076712,0.076712,0.076712,16.321634
2,13007.062646,NaN,-13007.062646,NaN,NaN,NaN,0.068269,0.068269,0.068269,14.525399


In [66]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,49999.999996,NaN,-49999.999996,NaN,NaN,NaN,0.131216,0.262432,50.0


As are generator limits:

In [67]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type,min_p_kw,max_p_kw,controllable
0,None,2,-80000.0,1.01,NaN,NaN,NaN,1.0,True,None,-80000.0,0.0,True
1,None,3,-100000.0,1.01,NaN,NaN,NaN,1.0,True,None,-100000.0,0.0,True


In [68]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-52377.399541,NaN,-3.845626,1.0
1,-37622.600463,NaN,-2.426876,1.0


The cost function is the same for the linearized OPF as for the non-linear one:

In [69]:
net.res_cost

96000.000003086927